In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import packages and libraries
#setting up initialization

In [2]:
#load the dataset into dataframe "df"
df = pd.read_csv("Building_Permits.csv", index_col = None)
df.head()

C:\Users\Anthony\Documents\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [3]:
#drop rows of more than 8 missing values
df.dropna(thresh=35, inplace =True)

#return new shape of the dataframe
df.shape

(100587, 43)

In [4]:
#drop columns with too many missing values
df.drop(columns=['Street Number Suffix', 'Unit', 'Unit Suffix','Structural Notification', 'Completed Date','Structural Notification','Voluntary Soft-Story Retrofit','Fire Only Permit','TIDF Compliance','Site Permit'], inplace = True)


In [5]:
#return total null value in each column
df.isnull().sum()

Permit Number                               0
Permit Type                                 0
Permit Type Definition                      0
Permit Creation Date                        0
Block                                       0
Lot                                         0
Street Number                               0
Street Name                                 0
Street Suffix                             514
Description                                 3
Current Status                              0
Current Status Date                         0
Filed Date                                  0
Issued Date                                 0
First Construction Document Date            2
Number of Existing Stories                  1
Number of Proposed Stories                  3
Permit Expiration Date                      1
Estimated Cost                             21
Revised Cost                                0
Existing Use                               10
Existing Units                    

#### In this prototype, we choose to use numerical attribute for the KNN classification, so we drop all categorical variable columns:

In [6]:
#drop columns wuth categorical values
df.drop(columns=['Permit Type Definition', 'Permit Creation Date', "Block", 'Description', "Lot", "Street Name", "Street Suffix", "Current Status", "Current Status Date", "Filed Date", "Issued Date", "First Construction Document Date", "Existing Use", "Proposed Use", "Existing Construction Type Description", "Proposed Construction Type Description", "Neighborhoods - Analysis Boundaries", "Location"], inplace = True)


In [7]:
#fill in null values by the mean value of eachx column
df.fillna(df.mean(), inplace = True)

In [8]:
df2 = pd.read_csv("Building_Permits.csv", index_col = None)
df2.dropna(thresh=35, inplace =True)
df2 = df2.loc[:, ['Filed Date','Issued Date']] #choose Filed Date and Issued Date columns only

C:\Users\Anthony\Documents\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df2.head()

,Filed Date,Issued Date
3,11/07/2016,07/18/2017
5,06/14/2017,07/06/2017
6,06/30/2017,06/30/2017
48,10/11/2017,10/11/2017
92,08/25/2014,08/25/2014


In [10]:
df2['Filed Date'] = pd.to_datetime(df2['Filed Date'])
df2['Issued Date'] = pd.to_datetime(df2['Issued Date'])

In [11]:
#New attribute necessary for my prediction
df2['Time'] = (df2['Issued Date'] - df2['Filed Date']) / np.timedelta64(1, 'D')

In [12]:
df0=pd.concat([df, df2], axis=1)

In [13]:
df0.head()

,Permit Number,Permit Type,Street Number,Number of Existing Stories,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Units,Proposed Units,Plansets,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Record ID,Filed Date,Issued Date,Time
3,201611072166,8,1230,2.0,2.0,07/13/2018,2000.0,2000.0,1.0,1.0,2.0,5.0,5.0,3.0,94109.0,1443574295566,2016-11-07,2017-07-18,253.0
5,201706149344,8,800,5.0,5.0,07/01/2018,4000.0,4000.0,326.0,326.0,2.0,1.0,1.0,10.0,94107.0,1466911170855,2017-06-14,2017-07-06,22.0
6,201706300814,8,1291,3.0,3.0,06/25/2018,12000.0,12000.0,5.0,5.0,0.0,5.0,5.0,5.0,94122.0,1468970403692,2017-06-30,2017-06-30,0.0
48,201710110928,8,938,2.0,2.0,04/11/2018,14000.0,14000.0,1.0,1.0,0.0,5.0,5.0,9.0,94134.0,1482932191681,2017-10-11,2017-10-11,0.0
92,201408254591,8,63,3.0,3.0,08/20/2015,500.0,500.0,4.0,4.0,0.0,5.0,5.0,6.0,94103.0,1353299454075,2014-08-25,2014-08-25,0.0


In [29]:
#categorize waiting time into bins
df0['cat_time'] = pd.cut(df0.Time, bins=[0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1350, 1500, 1800]
                                      , labels = ['0-149', '150-299', '300-449','450-599','600-749', 
                                                  '750-899', '900-1049', '1050-1199', '1200-1349', '1350-1500', '1500-1650']
                                      , right=False)

df0.head()

,Permit Number,Permit Type,Street Number,Number of Existing Stories,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Units,Proposed Units,Plansets,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Record ID,Filed Date,Issued Date,Time,cat_time
3,201611072166,8,1230,2.0,2.0,07/13/2018,2000.0,2000.0,1.0,1.0,2.0,5.0,5.0,3.0,94109.0,1443574295566,2016-11-07,2017-07-18,253.0,150-299
5,201706149344,8,800,5.0,5.0,07/01/2018,4000.0,4000.0,326.0,326.0,2.0,1.0,1.0,10.0,94107.0,1466911170855,2017-06-14,2017-07-06,22.0,0-149
6,201706300814,8,1291,3.0,3.0,06/25/2018,12000.0,12000.0,5.0,5.0,0.0,5.0,5.0,5.0,94122.0,1468970403692,2017-06-30,2017-06-30,0.0,0-149
48,201710110928,8,938,2.0,2.0,04/11/2018,14000.0,14000.0,1.0,1.0,0.0,5.0,5.0,9.0,94134.0,1482932191681,2017-10-11,2017-10-11,0.0,0-149
92,201408254591,8,63,3.0,3.0,08/20/2015,500.0,500.0,4.0,4.0,0.0,5.0,5.0,6.0,94103.0,1353299454075,2014-08-25,2014-08-25,0.0,0-149


In [30]:
dftest = df0[['Permit Type', 'Street Number', 'Number of Existing Stories', 
             'Number of Proposed Stories', 'Estimated Cost', 'Revised Cost', 
            'Proposed Units', 'Existing Construction Type', 'Proposed Construction Type',
            'Supervisor District', 'Zipcode', 'cat_time']]

dftest.columns

Index(['Permit Type', 'Street Number', 'Number of Existing Stories',
       'Number of Proposed Stories', 'Estimated Cost', 'Revised Cost',
       'Proposed Units', 'Existing Construction Type',
       'Proposed Construction Type', 'Supervisor District', 'Zipcode',
       'cat_time'],
      dtype='object')

In [31]:
dftest.isnull().sum()

Permit Type                   0
Street Number                 0
Number of Existing Stories    0
Number of Proposed Stories    0
Estimated Cost                0
Revised Cost                  0
Proposed Units                0
Existing Construction Type    0
Proposed Construction Type    0
Supervisor District           0
Zipcode                       0
cat_time                      0
dtype: int64

In [32]:
X = dftest.iloc[:, 0:11].values #attributes
y = dftest.iloc[:, -1].values #label

In [33]:
X[0]

array([8.0000e+00, 1.2300e+03, 2.0000e+00, 2.0000e+00, 2.0000e+03,
       2.0000e+03, 1.0000e+00, 5.0000e+00, 5.0000e+00, 3.0000e+00,
       9.4109e+04])

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [35]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
knn.score(X_train, y_train)
y_pred = knn.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Summary of the predictions made by the classifier
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
# Accuracy score
from sklearn.metrics import accuracy_score
print('accuracy is',accuracy_score(y_pred,y_test))

C:\Users\Anthony\Documents\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       0-149       0.95      0.99      0.97     18937
   1050-1199       0.00      0.00      0.00         7
   1200-1349       0.00      0.00      0.00         5
   1350-1500       0.00      0.00      0.00         1
     150-299       0.52      0.16      0.24       655
     300-449       0.17      0.03      0.05       276
     450-599       0.27      0.04      0.08       134
     600-749       0.21      0.06      0.09        52
     750-899       0.00      0.00      0.00        40
    900-1049       0.00      0.00      0.00        11

    accuracy                           0.94     20118
   macro avg       0.21      0.13      0.14     20118
weighted avg       0.91      0.94      0.92     20118

[[18804     0     0     0    78    33    14     4     4     0]
 [    6     0     0     0     1     0     0     0     0     0]
 [    5     0     0     0     0     0     0     0     0     0]
 [    1     0     0     0     0     0     0     0   